In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler  # Import LearningRateScheduler
from sklearn.model_selection import train_test_split
import os

# Define your dataset directory
dataset_dir = '/content/drive/MyDrive/SEMESTER 5/NST/SeabedObjects-KLSG--II'

# Define constants
input_size = (224, 224)
num_classes = len(os.listdir(dataset_dir))  # Number of classes in your dataset
batch_size = 32
epochs = 50

# Split your dataset into training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data will be used for validation
)

# Load your dataset using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # This will load the training set
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # This will load the validation set
)

# Load the VGG19 model with pre-trained weights (excluding the top classification layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))

# Add custom classification layers on top of the VGG19 base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Unfreeze the last 10 layers of the base model for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compile the model with the Adam optimizer and a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Implement a learning rate scheduler
def schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(schedule)

# Train the model with fine-tuning and learning rate scheduler
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {validation_accuracy * 100:.2f}%")


Found 545 images belonging to 4 classes.
Found 136 images belonging to 4 classes.
Epoch 1/50
18/18 [==============================] - 216s 11s/step - loss: 1.0982 - accuracy: 0.5174 - val_loss: 0.7168 - val_accuracy: 0.7574
Epoch 2/50
18/18 [==============================] - 14s 833ms/step - loss: 0.7750 - accuracy: 0.7413 - val_loss: 0.5453 - val_accuracy: 0.8382
Epoch 3/50
18/18 [==============================] - 14s 784ms/step - loss: 0.5388 - accuracy: 0.8459 - val_loss: 0.6928 - val_accuracy: 0.8162
Epoch 4/50
18/18 [==============================] - 14s 777ms/step - loss: 0.5227 - accuracy: 0.8165 - val_loss: 0.4171 - val_accuracy: 0.8529
Epoch 5/50
18/18 [==============================] - 14s 774ms/step - loss: 0.4196 - accuracy: 0.8642 - val_loss: 0.5840 - val_accuracy: 0.8162
Epoch 6/50
18/18 [==============================] - 15s 802ms/step - loss: 0.3678 - accuracy: 0.8679 - val_loss: 0.4818 - val_accuracy: 0.8162
Epoch 7/50
18/18 [==============================] - 14s 791ms

In [ ]:
import pickle

# Save the trained model to a file using pickle
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the input image and preprocess it
img_path = '/content/drive/MyDrive/SEMESTER 5/NST/SeabedObjects-KLSG--II/plane/plane-054.png'  # Replace with the path to your input image
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize the image

# Make a prediction
predictions = model.predict(img)

# Decode the predictions to get the class label
predicted_class = np.argmax(predictions)

print(f"Predicted class: {predicted_class}")


1/1 [==============================] - 0s 17ms/step
Predicted class: 2


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

# Define your dataset directory
dataset_dir = '/content/drive/MyDrive/SEMESTER 5/NST/SeabedObjects-KLSG--II'

# Define constants
input_size = (224, 224)
num_classes = len(os.listdir(dataset_dir))  # Number of classes in your dataset
batch_size = 32
epochs = 30  # Number of training epochs

# Create an ImageDataGenerator for rescaling the images
data_generator = ImageDataGenerator(rescale=1.0/255.0)

# Load your dataset using flow_from_directory
train_generator = data_generator.flow_from_directory(
    dataset_dir,  # Specify the path to your dataset
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)
X, y = train_generator.next()  # Extract features and labels from the generator

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and compile a simple model
model = Sequential()
model.add(Flatten(input_shape=(input_size[0], input_size[1], 3)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model without data augmentation
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Save the trained model for future use
model.save('my_model.keras')
